In [8]:
import pandas as pd
# from google.colab import drive
# drive.mount('/content/drive')

# Load AIS data (example CSV with vessel details and positions)
ais_data = pd.read_csv('AIS_2024_01_29.csv')

# Convert BaseDateTime to datetime format for time-based sorting
ais_data['BaseDateTime'] = pd.to_datetime(ais_data['BaseDateTime'])

# Sort by MMSI and BaseDateTime to analyze each vessel's movement sequentially
ais_data = ais_data.sort_values(['MMSI', 'BaseDateTime'])

# Detect anomalies in AIS data based on speed and course changes
def detect_anomalies(df):
    anomalies = []
    unique_vessels = df['MMSI'].unique()

    for vessel in unique_vessels:
        vessel_data = df[df['MMSI'] == vessel]

        for i in range(1, len(vessel_data)):
            speed_change = abs(vessel_data['SOG'].iloc[i] - vessel_data['SOG'].iloc[i - 1])
            course_change = abs(vessel_data['COG'].iloc[i] - vessel_data['COG'].iloc[i - 1])

            if speed_change > 10 or course_change > 30:
                anomalies.append(vessel_data.iloc[i])

    return pd.DataFrame(anomalies)

# Detect anomalies
anomalies_df = detect_anomalies(ais_data)
print("Anomalies detected:")
print(anomalies_df)


Anomalies detected:
              MMSI        BaseDateTime       LAT       LON  SOG    COG  \
11516            0 2024-01-29 00:01:17  43.64546 -70.24829  0.0  125.2   
15989            0 2024-01-29 00:02:27  43.64546 -70.24828  0.0  342.4   
19896            0 2024-01-29 00:03:36  43.64546 -70.24828  0.0   80.9   
161767           0 2024-01-29 00:04:45  43.64546 -70.24828  0.0  257.8   
39375            0 2024-01-29 00:08:05  43.64547 -70.24828  0.0    0.0   
...            ...                 ...       ...       ...  ...    ...   
4364752  998975541 2024-01-29 17:22:08  30.28763 -87.55561  0.0   62.5   
4436542  998975541 2024-01-29 17:37:38  30.28633 -87.55659  0.8  222.9   
4438504  998975541 2024-01-29 17:39:07  30.28638 -87.55654  0.2  133.8   
4443615  998975541 2024-01-29 17:40:38  30.28655 -87.55630  0.0   61.8   
6432492  998975541 2024-01-29 22:43:38  30.28476 -87.55924  0.0   82.6   

         Heading VesselName         IMO CallSign  VesselType  Status  Length  \
11516      

In [9]:
# Install necessary packages
!pip install geemap
!pip install earthengine-api
# Install Earth Engine and Geemap
!pip install earthengine-api geemap


# Authenticate and initialize Earth Engine
import ee
import geemap

# Trigger the authentication
ee.Authenticate()

# Initialize the Earth Engine API
ee.Initialize(project='ee-oilspilldetection-by-uday')


In [10]:
def plot_histogram(image, region, title):
    try:
        array = image.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=region,
            scale=30,
            maxPixels=1e9
        ).get('VV')

        # Check if array is empty
        if array is None:
            print("No VV data available for histogram plot.")
            return

        array = np.array(array.getInfo())
        plt.hist(array, bins=50, color='blue', alpha=0.7)
        plt.title(title)
        plt.xlabel('VV (dB)')
        plt.ylabel('Frequency')
        plt.grid()
        plt.show()
    except Exception as e:
        print(f"Error plotting histogram: {e}")

In [11]:
from datetime import datetime

# Function to parse date from BaseDateTime
def extract_date(base_date_time):
    # Convert to string if it’s a Timestamp
    if isinstance(base_date_time, pd.Timestamp):
        base_date_time = base_date_time.strftime('%Y-%m-%dT%H:%M:%S')
    else:
        base_date_time = str(base_date_time)

    # Parse the date
    date_time_obj = datetime.strptime(base_date_time, '%Y-%m-%dT%H:%M:%S')
    return date_time_obj.strftime('%Y-%m-%d')


In [12]:
def preprocess_image_for_model(image_np):
    """ Preprocess the NumPy array image to match model input requirements """
    # Resize the image to (256, 256)
    resized_image = cv2.resize(image_np, (256, 256))
    
    # Check the number of channels
    if resized_image.shape[-1] == 2:
        # Convert 2-channel input to 3-channel by duplicating one of the channels
        resized_image = np.repeat(resized_image[:, :, :1], 3, axis=-1)
    elif resized_image.shape[-1] == 1:
        # Convert 1-channel input to 3-channel
        resized_image = np.repeat(resized_image, 3, axis=-1)
    elif resized_image.shape[-1] != 3:
        raise ValueError(f"Unexpected number of channels: {resized_image.shape[-1]}")
    
    # Add a batch dimension
    return np.expand_dims(resized_image, axis=0)


In [13]:
import tensorflow as tf

# Define jaccard_coef if not already defined
def jaccard_coef(y_true, y_pred):
    y_true_flatten = tf.reshape(y_true, [-1])
    y_pred_flatten = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_flatten * y_pred_flatten)
    sum_ = tf.reduce_sum(y_true_flatten) + tf.reduce_sum(y_pred_flatten)
    smooth = 1e-6  # To avoid division by zero
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac


In [ ]:
import tensorflow as tf
import segmentation_models as sm
from tensorflow.keras.models import load_model

# Define loss functions
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.CategoricalFocalLoss()

# Custom objects dictionary
custom_objects = {
    "jaccard_coef": jaccard_coef,
    "DiceLoss": dice_loss,
    "CategoricalFocalLoss": focal_loss,
}

# Load models
hybrid_model = load_model(r"C:\Users\Uday Kiran Ambati\OneDrive\Desktop\major_project\Code\saved_models\hybrid_model.h5")
unet_model = load_model(r"C:\Users\Uday Kiran Ambati\OneDrive\Desktop\major_project\Code\saved_models\U_net.h5")
deeplab_model = load_model(
    r"C:\Users\Uday Kiran Ambati\OneDrive\Desktop\major_project\Code\saved_models\deeplab_model.h5",
    custom_objects=custom_objects
)

TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 256, 256, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_1'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']

In [ ]:
def confirm_oil_spill(hybrid_model, unet_model, deeplab_model, image):
    """ 
    Confirm oil spill detection using Hybrid, UNet, and DeepLabV3+ models. 
    Requires agreement among all models.
    """
    processed_image = preprocess_image_for_model(image)

    # Get predictions from all three models
    hybrid_prediction = hybrid_model.predict(processed_image)
    unet_prediction = unet_model.predict(processed_image)
    deeplab_prediction = deeplab_model.predict(processed_image)

    # Assuming '1' corresponds to oil spill detection
    hybrid_result = np.argmax(hybrid_prediction) == 1
    unet_result = np.argmax(unet_prediction) == 1
    deeplab_result = np.argmax(deeplab_prediction, axis=-1).squeeze() == 1

    # Confirm only if all models agree
    return hybrid_result and unet_result and deeplab_result


In [ ]:
from tensorflow.keras.models import load_model
import geemap
import ee
import numpy as np
import cv2
from IPython.display import display
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_custom_objects
def process_anomaly(row):
    lat = row['LAT']
    lon = row['LON']
    base_date_time = row['BaseDateTime']
    date = extract_date(base_date_time)
    anomaly_date = ee.Date(date)
    print(f"Processing anomaly at {lat}, {lon} on {date}")

    roi = ee.Geometry.Point(lon, lat).buffer(10000).bounds()
    detected_spills = []

    sen1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
        .filterDate(anomaly_date, anomaly_date.advance(1, 'day')) \
        .filterBounds(roi) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW'))

    if sen1.size().getInfo() == 0:
        print("No Sentinel-1 VV or VH images available. Trying Sentinel-2.")
        sen1 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
            .filterDate(anomaly_date, anomaly_date.advance(1, 'day')) \
            .filterBounds(roi) \
            .filter(ee.Filter.eq('CLOUDY_PIXEL_PERCENTAGE', 0))

        if sen1.size().getInfo() == 0:
            print("No Sentinel-2 images available for this date and location.")
            return

        sen1_image = sen1.select(['B4', 'B8']).mosaic()
    else:
        sen1_image = sen1.select(['VV', 'VH']).mosaic()

    despeckled = sen1_image.focal_mean(100, 'square', 'meters')
    vv_dark = despeckled.select('VV').lt(-22)
    vh_dark = despeckled.select('VH').lt(-18)
    oil_spill_detection = vv_dark.And(vh_dark)

    mask = oil_spill_detection.updateMask(oil_spill_detection)
    area = mask.multiply(ee.Image.pixelArea().divide(1e6))
    oil_spill_area = ee.Number(
        area.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=roi,
            scale=100
        ).values().get(0)
    ).getInfo()

    if oil_spill_area > 0:
        print(f"Detected Oil Spill Area (sq. km): {oil_spill_area}")

        despeckled_np = geemap.ee_to_numpy(despeckled, bands=['VV', 'VH'], region=roi, scale=100)
        processed_image = preprocess_image_for_model(despeckled_np)

# Confirm using all three models
#if confirm_oil_spill(hybrid_model, unet_model, deeplab_model, processed_image):
 #   detected_spills.append(oil_spill_area)

        #hybrid_model = load_model(r"C:\Users\harsh\OneDrive\Desktop\Capstone\hybrid_model.h5")
        #unet_model = load_model(r"C:\Users\harsh\OneDrive\Desktop\Capstone\U_net.h5")
        # If using custom layers/functions
        # deeplab_model = load_model(
        #     r"C:\Users\harsh\OneDrive\Desktop\Capstone\deeplab_model.h5",
        #     custom_objects=get_custom_objects()
        # )

        # Confirm using all three models
        if confirm_oil_spill(hybrid_model, unet_model, deeplab_model, despeckled_np):
            detected_spills.append(oil_spill_area)

            oil_spill_vis_params = {
                'min': 0,
                'max': 1,
                'palette': ['blue', 'cyan', 'yellow', 'red']
            }

            Map = geemap.Map()
            Map.centerObject(roi)
            Map.addLayer(sen1_image, {'bands': ['VV', 'VH'], 'min': -30, 'max': 0}, f'Sentinel-1 Image {anomaly_date.getInfo()}')
            Map.addLayer(despeckled.clip(roi), {}, 'Despeckled Image', False)
            Map.addLayer(oil_spill_detection.clip(roi), {}, 'Oil Spill Detection', False)
            oil_spill_vector = mask.reduceToVectors(
                geometry=roi,
                scale=100
            )
            Map.addLayer(oil_spill_vector, {}, 'Oil Spill Vector', False)

    if detected_spills:
        print(f"Oil spills detected over the days: {detected_spills}")
        Map.addLayerControl()
        return Map
    else:
        print("No significant oil spills detected across the checked dates.")
        return None


In [ ]:
# Iterate over each anomaly in the DataFrame and process
for index, row in anomalies_df.iterrows():
    result_map = process_anomaly(row)
    if result_map:
        display(result_map)  # This displays the map in interactive environments

Processing anomaly at 25.78166, -80.15023 on 2024-01-01
Detected Oil Spill Area (sq. km): 178.70174741171064
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 904ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 832ms/step
No significant oil spills detected across the checked dates.
Processing anomaly at 27.78969, -97.39082 on 2024-01-01
No Sentinel-1 VV or VH images available. Trying Sentinel-2.
No Sentinel-2 images available for this date and location.
Processing anomaly at 32.02417, -81.04617 on 2024-01-01
Detected Oil Spill Area (sq. km): 17.5633537601677
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
No significant oil spills detected across the checked dates.
Processing anomaly at 47.98236, -122.21983 on 2024-01-01
No Sentinel-1 VV or VH images available. Trying Sentinel-2.
No Sentinel-2 images available for this date and location.
Processing anomaly at 28.17053, -89.22213 on 2024-01-01
No Sentinel-1 VV or VH 